In [11]:
import cv2
import numpy as np

cap = cv2.VideoCapture("./sam.avi")
ret, ref_frame = cap.read()
# ref_frame=cv2.resize(ref_frame,(640,420))
prev_gray = cv2.cvtColor(ref_frame, cv2.COLOR_BGR2GRAY)
mask = np.zeros_like(ref_frame)
mask[..., 1] = 255
print(mask.shape,"<<")
ref_img=cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
# print(ref_img.shape,"//")

fw = int(cap.get(3))
fh = int(cap.get(4))
# videosave = cv2.VideoWriter('./output/demo.mp4', cv2.VideoWriter_fourcc(*'XVID'), 15, (fw, fh),True)


def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

while(cap.isOpened()):
    ret, frame = cap.read()
    
    if not ret: 
        break
    h,w,_= frame.shape    
#     frame=cv2.resize(frame,(640,420))
#     cv2.imshow("input", frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    mask[..., 0] = angle * 180 / np.pi / 2
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    compare_img=cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
    if rgb.sum() >=1190000: #threshold should be fixed manually
        cv2.putText(rgb, "Motion Detected", (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
        
    
    final=stackImages(0.8,([frame,rgb]))
    frame=cv2.resize(frame,(fw,fh))
#     videosave.write(final)
    cv2.imshow("dense optical flow", final)
    prev_gray = gray
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
# videosave.release()
cv2.destroyAllWindows()


(720, 1280, 3) <<
